In [1]:
import tushare as ts
import plotly
import plotly.graph_objs as go

ts.set_token('876d6d6b5774b023ceea5d01b4ecbd49f64d7c9e22d0661c87d7a8fb')
pro = ts.pro_api()
plotly.offline.init_notebook_mode(connected=True)

In [7]:
def date_format_transfer(date, split_simbol='-'):
    return ["{}{}{}{}{}".format(a[:4], split_simbol, a[4:6], split_simbol, a[6:8]) for a in date]

In [ ]:
daily = pro.daily(ts_code='002236.SZ', start_date='20180627', end_date='20181128')
turnover_rate = pro.daily_basic(ts_code='002236.SZ', start_date='20180627', end_date='20181128')
date = date_format_transfer(daily.trade_date)

open_line = go.Scatter(
                x = date,
                y=daily.open,
                name = "open",
                line = dict(color = '#FF0000'),
                opacity = 0.8)

close_line = go.Scatter(
                x = date,
                y=daily.close,
                name = "close",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

tunrover_line = go.Scatter(
                x = date,
                y=turnover_rate.turnover_rate,
                name = "turnover rate",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

data = [open_line, close_line, tunrover_line]
plotly.offline.iplot(data)

In [ ]:
df = ts.pro_bar(pro_api=pro, ts_code='002236.SZ', start_date='20181120', end_date='20181128')
dfq = ts.pro_bar(pro_api=pro,ts_code='002236.SZ', adj='qfq', start_date='20181120', end_date='20181128')
dfh = ts.pro_bar(pro_api=pro,ts_code='002236.SZ', adj='hfq', start_date='20181120', end_date='20181128')
print(df)
print(dfq)
print(dfh)
